In [2]:
# Preprocessing file names for the dataset
import os

def rename_files_in_folder(folder_path, GEN):
    # Get all file names in the specified folder
    files = os.listdir(folder_path)
    
    # Filter out directories, keep only files
    files = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]
    
    # Sort files alphabetically before renaming to maintain some order
    files.sort()
    
    # Loop through sorted files and rename them
    for index, file_name in enumerate(files):
        # Get the extension of the file
        extension = os.path.splitext(file_name)[1]
        
        # Define new file name with the same extension
        if GEN:
            new_file_name = f"gen{index + 1}{extension}"
        else:
            new_file_name = f"real{index + 1}{extension}"
        
        # Create full file paths
        old_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(folder_path, new_file_name)
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        
        # print(f"Renamed '{file_name}' to '{new_file_name}'")

generated_images = "E:\\CS 512 Project\\Proj\\data_image_classification\\generated_images"
real_images = "E:\\CS 512 Project\\Proj\\data_image_classification\\real_images"

In [3]:
rename_files_in_folder(generated_images, True)
rename_files_in_folder(real_images, False)